# Animations and Tidy Data
If you want to type along with me, use [this notebook](https://humboldt.cloudbank.2i2c.cloud/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fbethanyj0%2Fdata271_sp25&branch=main&urlpath=tree%2Fdata271_sp25%2Flectures%2Fdata271_lec35_live.ipynb) instead. 
If you don't want to type and want to follow along just by executing the cells, stay in this notebook. 

In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from ipywidgets import interact
import ipywidgets as widgets

### Animations/sliders with plotly

In [ ]:
# Import the gap minder
df = pd.read_csv('gapminder_all.csv')
df

In [ ]:
# Tidy the data (start by melting)
long_df = df.melt(id_vars=['continent','country'])
long_df

In [ ]:
# Each variable gets its own column
long_df[['var','year']] = long_df.variable.str.split('_',expand=True)
long_df.drop(columns = 'variable',inplace=True)
long_df

In [ ]:
# pivot so each variable is a column
tidy_df = long_df.pivot_table(index = ['continent','country','year'],
                              columns='var',values='value').reset_index()
tidy_df

In [ ]:
# check info for datatypes and nulls
tidy_df.info()

In [ ]:
# cast year to type int
tidy_df['year'] = tidy_df.year.astype(int)

### How would we visualize with Seaborn?

In [ ]:
# scatterplot in seaborn for one year
sns.scatterplot(data = tidy_df[tidy_df['year']== 2007],
                x = 'gdpPercap',y='lifeExp',size='pop', 
                sizes=(25, 250), hue='continent',alpha=0.7)
plt.xscale('log')

In [ ]:
# Show the trend across years
sns.relplot(data = tidy_df,x = 'gdpPercap',y='lifeExp',
            size='pop', sizes=(25, 250), 
            hue='continent',alpha=0.7,
            col='year',col_wrap=4)
plt.xscale('log')

### Same data with interactive plots

In [ ]:
# Look at just one year
fig = px.scatter(tidy_df[tidy_df['year']==2007], x="gdpPercap", y="lifeExp",
           size="pop", color="continent", 
           log_x=True, size_max=55,
           hover_name='country')
fig.show()

In [ ]:
# Create animation/slider instead of facetting
fig = px.scatter(tidy_df, x="gdpPercap", y="lifeExp",
           size="pop", color="continent", 
           hover_name="country",
           animation_frame='year',
           log_x=True, size_max=55, range_x=[100,100000], range_y=[25,90])
fig.show()

In [ ]:
# Combine animations with dropdowns
def update_plot(option):
    if option:
        x_scale = [100,100000]
    else:
        x_scale = [0,45000]
    fig = px.scatter(tidy_df, x="gdpPercap", y="lifeExp",
           size="pop", color="continent", 
           hover_name="country",
           animation_frame="year", 
           log_x=option, size_max=55, range_x=x_scale, range_y=[25,90])
    fig.show()

# Define dropdown options for x and y axes
scale_options = [True, False]

# Create dropdown widgets
scale_dropdown = widgets.Dropdown(options=scale_options, description='Log scale:')

# Create interactive plot using ipywidgets interact function
interact(update_plot, option=scale_dropdown);

In [ ]:
# Can also make animated bar charts
fig = px.bar(tidy_df, x="continent", y="pop", color="continent",
              animation_frame="year", hover_name = 'country',range_y=[0,4000000000])
fig.show()